# Imports

In [1]:
import os
import sys
import faiss
import time
import logging
import numpy as np
import pickle
import json
from typing import List, Dict, Any

In [2]:
import faiss
#from sentence_transformers import SentenceTransformer

# Carrega embeddings

In [2]:
from scripts.preprocess_embeddings import load_config, load_models
config = load_config("configs")
processor, model, device = load_models(config)

IsADirectoryError: [Errno 21] Is a directory: 'configs'

In [3]:
# Configuração de diretórios
EMBEDDINGS_DIR = "artifacts/embeddings"
VECTOR_STORE_DIR = "artifacts/vector_store"

# Criar diretórios se não existirem
os.makedirs(VECTOR_STORE_DIR, exist_ok=True)

print(f"📁 Diretório de embeddings: {EMBEDDINGS_DIR}")
print(f"📁 Diretório de vector store: {VECTOR_STORE_DIR}")

# Verificar se o arquivo existe
study_ids_file = os.path.join(EMBEDDINGS_DIR, "study_ids.npy")
print(f"📄 Arquivo alvo: {study_ids_file}")
print(f"📊 Arquivo existe: {os.path.exists(study_ids_file)}")

📁 Diretório de embeddings: artifacts/embeddings
📁 Diretório de vector store: artifacts/vector_store
📄 Arquivo alvo: artifacts/embeddings/study_ids.npy
📊 Arquivo existe: True


In [4]:
def load_embeddings(embeddings_file: str) -> np.ndarray:
    """
    Carrega os embeddings salvos
    """
    
    study_ids_file = os.path.join(EMBEDDINGS_DIR, embeddings_file)
    
    if not os.path.exists(study_ids_file):
        print(f"❌ Arquivo não encontrado: {study_ids_file}")
        return None
    
    try:
        # Carregar o arquivo numpy
        embeddings = np.load(study_ids_file, allow_pickle=True)
        
        print(f"✅ Embeddings carregados com sucesso!")
        print(f"📊 Formato dos dados: {type(embeddings)}")
        print(f"📊 Shape: {embeddings.shape if hasattr(embeddings, 'shape') else 'N/A'}")
        
        return embeddings
        
    except Exception as e:
        print(f"❌ Erro ao carregar embeddings: {e}")
        return None

In [5]:
study_ids = load_embeddings("study_ids.npy")

✅ Embeddings carregados com sucesso!
📊 Formato dos dados: <class 'numpy.ndarray'>
📊 Shape: (227835,)


In [6]:
study_emb = load_embeddings("e_study.npy") 

✅ Embeddings carregados com sucesso!
📊 Formato dos dados: <class 'numpy.ndarray'>
📊 Shape: (227835, 1152)


In [7]:
images_emb = load_embeddings("e_img.npy") 

✅ Embeddings carregados com sucesso!
📊 Formato dos dados: <class 'numpy.ndarray'>
📊 Shape: (227835, 1152)


In [8]:
text_emb = load_embeddings("e_text.npy")

✅ Embeddings carregados com sucesso!
📊 Formato dos dados: <class 'numpy.ndarray'>
📊 Shape: (227835, 1152)


# Cria vector store

In [14]:
# Criar vector store FAISS com os embeddings carregados
print("🔧 Criando vector store FAISS...")

# Verificar se os embeddings foram carregados corretamente
if study_emb is not None and study_ids is not None:
    print(f"📊 Número de embeddings: {len(study_emb)}")
    print(f"📊 Dimensionalidade: {study_emb.shape[1]}")
    print(f"📊 Número de IDs: {len(study_ids)}")
    
    # Criar FAISS index diretamente
    dimension = study_emb.shape[1]
    index = faiss.IndexFlatIP(dimension)  # Inner Product para similaridade de cosseno
    
    # Normalizar embeddings para usar inner product como similaridade de cosseno
    faiss.normalize_L2(study_emb)
    
    # Adicionar embeddings ao index
    index.add(study_emb)
    
    # Salva vector store 
    INDEX_FILE = os.path.join(VECTOR_STORE_DIR, "faiss_study.index")
    faiss.write_index(index, INDEX_FILE)
    
    print("✅ Vector store criado com sucesso!")
    print(f"📊 Total de vetores no índice: {index.ntotal}")
    
else:
    print("❌ Erro: Embeddings não carregados corretamente")

🔧 Criando vector store FAISS...
📊 Número de embeddings: 227835
📊 Dimensionalidade: 1152
📊 Número de IDs: 227835
✅ Vector store criado com sucesso!
📊 Total de vetores no índice: 227835


In [9]:
def create_vector_store(
        embeddings: np.ndarray, 
        index_file: str,
        vector_store_dir=VECTOR_STORE_DIR
    ) -> faiss.Index:
    """
    Cria um índice Faiss a partir dos embeddings o fornecidos
    """
    
    # Criar vector store FAISS com os embeddings carregados
    print("🔧 Criando vector store FAISS...")

    # Verificar se os embeddings foram carregados corretamente
    if embeddings is not None:
        print(f"📊 Número de embeddings: {embeddings.shape[0]}")
        print(f"📊 Dimensionalidade: {embeddings.shape[1]}")
    
        # Criar FAISS index diretamente
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatIP(dimension)  # Inner Product para similaridade de cosseno

        # Normalizar embeddings para usar inner product como similaridade de cosseno
        faiss.normalize_L2(embeddings)

        # Adicionar embeddings ao index
        index.add(embeddings)

        # Salva vector store 
        INDEX_FILE = os.path.join(vector_store_dir, index_file)
        faiss.write_index(index, INDEX_FILE)

        print("✅ Vector store criado com sucesso!")
        print(f"📄 Arquivo de índice: {INDEX_FILE}")
        print(f"📊 Total de vetores no índice: {index.ntotal}")
    
    else:
        print("❌ Erro: Embeddings não carregados corretamente")

In [10]:
create_vector_store(images_emb, "faiss_img_aux.index")

🔧 Criando vector store FAISS...
📊 Número de embeddings: 227835
📊 Dimensionalidade: 1152
✅ Vector store criado com sucesso!
📄 Arquivo de índice: artifacts/vector_store/faiss_img_aux.index
📊 Total de vetores no índice: 227835


In [11]:
create_vector_store(text_emb, "faiss_txt_aux.index")

🔧 Criando vector store FAISS...
📊 Número de embeddings: 227835
📊 Dimensionalidade: 1152
✅ Vector store criado com sucesso!
📄 Arquivo de índice: artifacts/vector_store/faiss_txt_aux.index
📊 Total de vetores no índice: 227835


# Teste de busca

In [1]:
def extract_embedding_single_study(study_txt, study_imgs):
    """
    Extrai os embeddings de um único estudo
    Args:
        study_txt: str - Texto do estudo
        study_imgs: list - Lista de imagens do estudo
    """

    # === 1) Extrai o texto (Findings)
    findings = self._extract_findings(study_txt)

    tokenized_text = None
    if self.tokenizer is not None:
        tokenized_text = self.tokenizer(
            text=findings,
            truncation=True,
            padding="max_length",
            max_length=self.max_tokens,
            return_tensors="pt"
        )

    # === 2) Carrega e processa as imagens
    images_tensor = torch.stack(imgs)



## Busca

In [ ]:
def search_relevant_study(query_study, vector_store, study_ids, k=3):
    """"
    Pesquisa os estudos mais relevantes para uma determinada consulta.
    Args:
        - query_study: estudo de consulta já vetorizado
        - vector_store: vector store
        - k: número de estudos mais relevantes a serem retornados
    """
    # Preparar o vetor de consulta
    if isinstance(query_study, np.ndarray):
        # Garantir que é 2D e float32
        if query_study.ndim == 1:
            query_embedding = query_study.reshape(1, -1).astype(np.float32)
        else:
            query_embedding = query_study.astype(np.float32)
    else:
        query_embedding = np.array(query_study, dtype=np.float32).reshape(1, -1)
    
    # Normalizar o vetor de consulta
    faiss.normalize_L2(query_embedding)

    # Pesquisar os estudos mais relevantes
    D, I = vector_store.search(query_embedding, k)

    # Retornar os IDs dos estudos mais relevantes
    results = []
    for idx in I[0]:
        results.append(study_ids[idx])
        
    return results, I[0]

In [ ]:
ex_estudo = study_emb[0]

In [ ]:
ex_estudo.shape

In [ ]:
study_ids[0]

In [ ]:
paciente_emb[0]

In [ ]:
vector_store_study = faiss.read_index("artifacts/vector_store/faiss_study.index")  # Carregar o índice

In [ ]:
estudos, idxs = search_relevant_study(ex_estudo, vector_store_study,study_ids, k=4)  # Exemplo de uso

In [ ]:
search_relevant_study(ex_estudo, vector_store_study,study_ids, k=2)  # Exemplo de uso

In [ ]:
estudos

In [ ]:
idxs

In [ ]:
caminho = "../dados/mimic/files" 

ex1_img = caminho + '/p13/p13590440/s53634677'
ex1_txt = caminho + '/p13/p13590440/s53634677.txt'

In [ ]:
with open(ex1_txt, 'r') as file:
    content = file.read()
    print(content)

In [ ]:
import os
from PIL import Image

# Listar arquivos .jpg no diretório
jpg_files = []
if os.path.exists(ex1_img):
    for file in os.listdir(ex1_img):
        if file.lower().endswith('.jpg'):
            jpg_files.append(os.path.join(ex1_img, file))

print(f"Arquivos JPG encontrados: {len(jpg_files)}")
for jpg_file in jpg_files:
    print(f"📄 {jpg_file}")

# Carregar e exibir as imagens
imgs = []
for jpg_file in jpg_files:
    try:
        img = Image.open(jpg_file)
        imgs.append(img)
        print(f"✅ Imagem carregada: {os.path.basename(jpg_file)}")
        display(img)
    except Exception as e:
        print(f"❌ Erro ao carregar {jpg_file}: {e}")

In [ ]:
ex2 = idxs[2]
ex2_txt = caminho + '/p12/p12480689/s52231919.txt'
ex2_img = caminho + '/p12/p12480689/s52231919'

In [ ]:
with open(ex2_txt, 'r') as file:
    content = file.read()
    print(content)

In [ ]:
# Listar arquivos .jpg no diretório
jpg_files = []
if os.path.exists(ex2_img):
    for file in os.listdir(ex2_img):
        if file.lower().endswith('.jpg'):
            jpg_files.append(os.path.join(ex2_img, file))

print(f"Arquivos JPG encontrados: {len(jpg_files)}")
for jpg_file in jpg_files:
    print(f"📄 {jpg_file}")

# Carregar e exibir as imagens
imgs = []
for jpg_file in jpg_files:
    try:
        img = Image.open(jpg_file)
        imgs.append(img)
        print(f"✅ Imagem carregada: {os.path.basename(jpg_file)}")
        display(img)
    except Exception as e:
        print(f"❌ Erro ao carregar {jpg_file}: {e}")

In [ ]:
import pandas as pd

In [ ]:
labels = pd.read_csv("../dados/mimic/mimic-cxr-2.0.0-chexpert.csv")
labels.head()  # Exibir as primeiras linhas do DataFrame

In [ ]:
labels.shape

In [ ]:
labels.loc[
    labels['study_id'].isin([50414267,53634677]),
]

In [ ]:
labels['No Finding'].sum()/labels.shape[0]